<a href="https://colab.research.google.com/github/wesszabo/Analizis-2-informatikusoknak/blob/Gyakorlat/MultiVariable_extreme_value_gyak_feladatok.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

>[Table of Contents](#scrollTo=uSvfRB-3kuYe)

>[Szélsőérték számítás](#scrollTo=s0fYjtEIktby)

>[FELADATOK](#scrollTo=la6ckSZ5y_BT)

>>[Feladat 1. Legyen $f(x,y):=4x^2+9y^2+8x-36y+24$. Határozzuk meg $f$ lokális szélsőértékeit.](#scrollTo=JNE8E_oZzDPm)

>>[Feladatok 2. Legyen $f(x,y):=\dfrac{1}{3}x^3+y^2+2xy-6x-3y+4$. Határozzuk meg $f$ lokális szélsőértékeit.](#scrollTo=Ta81LxJp0J7G)

>>[Feladat 3. Legyen $f(x,y):=x^3+2xy-6x-4y^2$. Határozzuk meg $f$ lokális szélsőértékeit.](#scrollTo=uefleBJ000J_)

>>[Feladat 4. Legyen $f(x,y):=2xy+3x+4y$. Határozzuk meg $f$ lokális szélsőértékeit.](#scrollTo=qjrAKU-E1QZN)

>>[Feladat 5. Legyen $f(x,y):=3x^2-2xy+y^2-8y$. Határozzuk meg $f$ lokális szélsőértékeit.](#scrollTo=JEMkIh4r2UTc)

>>[Feladat 6. Legyen $f(x,y):=y^2+xy+3y+2x+3$. Határozzuk meg $f$ lokális szélsőértékeit.](#scrollTo=fe97AEj63lD8)

>>[Feladat 7. Legyen $f(x,y):=x^2+2y^2-x^2y$. Határozzuk meg $f$ lokális szélsőértékeit.](#scrollTo=Yb_MN8T86hxN)

>>[Feladat 8. Legyen $f(x,y):=x^2+10xy+y^2$. Határozzuk meg $f$ lokális szélsőértékeit.](#scrollTo=hHXeHpK07bsb)



In [0]:
###################  MODULOK 

In [0]:
from sympy import *
ip = get_ipython()
ibe = ip.configurables[-1]
ibe.figure_formats = { 'pdf', 'png'}

In [0]:
%matplotlib inline
import matplotlib.pyplot as plt
from mpl_toolkits import mplot3d
from IPython.display import HTML,Latex, display
import requests
import numpy as np
import sympy as sym
from sympy.solvers.solveset import solveset_real, nonlinsolve
from sympy import exp, log, sin, cos, sqrt, Matrix, Function, limit, diff,\
symbols, hessian, latex, simplify
from scipy import optimize
#from numpy import pi, sin, cos
from numpy import linalg
import plotly.graph_objects as go
from chart_studio.plotly import plot, iplot

In [0]:
import plotly.io as pio
pio.renderers.default = 'colab'

In [0]:
#####################  BEÁLLÍTÁSOK 

In [0]:
# Sympy to print formula not TeX
from sympy import init_printing
def custom_latex_printer(exp,**options):
    from google.colab.output._publish import javascript
    url = "https://cdnjs.cloudflare.com/ajax/libs/mathjax/2.7.3/latest.js?config=default"
    javascript(url=url)
    return sym.printing.latex(exp,**options)
init_printing(use_latex="mathjax",latex_printer=custom_latex_printer)

In [0]:
x, y, z, alpha, beta = symbols('x y z alpha beta', real = True)
a, m, r, theta = symbols('a m r theta', real = True) # parameters
from sympy.abc import rho, phi

In [0]:
# Jupyter to print the results for any variable or statement on it’s own line

In [0]:
from IPython.core.interactiveshell import InteractiveShell
InteractiveShell.ast_node_interactivity = "all"

In [0]:
############## TeX macros

$\newcommand{\grad}{\mathrm{grad}}$
$\newcommand{\vec}[1]{\mathbf{#1}}$

# Table of Contents

# Szélsőérték számítás

In [0]:
##############   SEGÉDFÜGGVÉNYEK

In [0]:
##############  GRADIENT 

In [0]:
#@title
def gradient(func, varlist):
    grad = [diff(func, var) for var in varlist]
    return grad        

In [0]:
#############  GRAFIKA

In [0]:
def draw_surface(func, x0=0.0, y0=0.0):
    xx = np.linspace(x0 - 3, x0 + 3, 100)
    yy = np.linspace(y0 - 3, y0 + 3, 100)
    x0_space = np.linspace(x0,x0,1)
    y0_space = np.linspace(y0,y0,1)
    X, Y = np.meshgrid(xx,yy)
    func_lambda = lambdify((x, y), func, modules='numpy')
    Z = func_lambda(X, Y)
    zz = func_lambda(x0_space, y0_space)
    global fig 
    fig = ['NaN']
    fig[0] = go.Figure(data=[
        go.Scatter3d(
        x = x0_space, y = y0_space,
        z = zz,
        #name = 'kritikus pont',
        marker = dict(
            size = 6,
            opacity=1.0
        ),
        line = dict(
            color = 'red',
            width = 4
        )
        ),
        go.Surface(
            x=xx, y=yy, z=Z,
            contours_z=dict(show=True, 
                            usecolormap=True,
                            highlightcolor="limegreen", 
                            project_z=True)
            )
        ])
    func_latex = '$\displaystyle{f(x,y)='+latex(func)+'}$'
    fig[0].update_layout(title=dict(
                    text = func_latex,
                    font=dict(size=20)),
                    scene_camera_eye=dict(x=1.87, y=0.88, z=0.64),
                    width=600, height=600, 
                    margin=dict(l=65, r=80, b=65, t=70),
                    legend=go.layout.Legend(
            x=0,
            y=1,
            traceorder="normal",
            font=dict(
                family="sans-serif",
                size=16,
                color="black"
            ),
            bgcolor="LightSteelBlue",
            bordercolor="Black",
            borderwidth=2
        )
    )
    fig[0].show(renderer="notebook_connected")

In [0]:
##############  SZÉLSŐÉRTÉK KERESÉS

In [0]:
#func = x**3/3 +y**2 + 2 * x * y - 6 * x - 3 * y + 4
#func = 2 * x * y + 3 * x + 4 * y
#func = x**3 + 2 * x * y - 6 * x - 4 * y**2
#func = x**2 + y**3 -x*y+ z**2
#func = x**2 -y**3
func = 4*x**2+9*y**2+8*x-36*y+24
func_latex = latex(func)
varlist = [x, y]
dim = len(varlist)
grad_list = gradient(func, varlist)
grad_func = Matrix(grad_list)
print(f' '+func_latex+ ' szélsőértékei')
print("grad(f) : ")
grad_func
Eq_list = ['NaN' for _ in range(dim)]
roots = list(nonlinsolve(grad_list, varlist))
if len(roots) == 0:
    print(f'A '+func_latex+ ' függvénynek nincs lokális szélsőértéke')
    draw_surface(func)
    print("\n")
    fig[0] = ['NaN']
    exit
hesse_matrix = hessian(func, varlist)
print("Hess(f) : ")
hesse_matrix
print("A")
Eq_list = ['NaN' for _ in range(dim)]
for i in range(dim):
    Eq_list[i] = Eq(grad_func[i])
    Eq_list[i]
print("egyenletrendszerből adódik, hogy")
number_of_roots = len(roots)
for j in range(number_of_roots):
    print(f'\n {j + 1}. kritikus pont: ')
    for k in range(dim):
        print(f'               {varlist[k]} = {roots[j][k]}')
    print('\nA Hesse mátrix a kritikus pontban:')
    hesse = hesse_matrix.subs([(varlist[k], roots[j][k]) for k in range(len(varlist))])
    hesse
    print("A Hesse mátrix aldeterminánsai: ")
    aldet = ([hesse[0:m, 0:m].det() for m in range(1, dim+1)])
    aldet
    print("Előjelei: ")
    aldet_sig_num = np.sign(aldet)
    aldet_sig_str = ["Nan" for _ in range(dim) ]
    for m in range(0,dim):
        if aldet_sig_num[m] == -1:
            aldet_sig_str[m] = "-"
        elif aldet_sig_num[m] == 1:
            aldet_sig_str[m] = "+"
        else:
            aldet_sig_str[m] = "0"
    print("            ", aldet_sig_str)
    aldet_sig_str_odd = aldet_sig_str[0::2]
    aldet_sig_str_even = aldet_sig_str[1::2]
    if sum(aldet_sig_num) == dim:
        print(f'\n A ' +func_latex+ ' függvénynek lokális minimuma van.\n')
        draw_surface(func, float(roots[j][0]), float(roots[j][1]))
        print("\n")
        fig[0] = ['NaN']
    elif (aldet_sig_str_odd.count("-") == len(aldet_sig_str_odd)) \
        and (aldet_sig_str_even.count("+") == len(aldet_sig_str_even)):
        print(f'\n A ' +func_latex+ ' függvénynek lokális maximuma van.\n')
        draw_surface(func, float(roots[j][0]), float(roots[j][1]))
        print("\n")
        fig[0] = ['NaN']
    elif ("0" not in aldet_sig_str):
        print(f'\n A ' +func_latex+ ' függvénynek nyeregpontja van.\n')
        draw_surface(func, float(roots[j][0]), float(roots[j][1]))
        print("\n")
        fig[0] = ['NaN']
    else:
        hesse_float = np.array(hesse).astype(np.float64)
        eigvalues = np.linalg.eigvalsh(hesse_float)        
        eigvalues_sig = np.sign(eigvalues)
        if (-1 in eigvalues_sig) and (1 in eigvalues_sig):
            print(f'\n A ' +func_latex+ ' függvénynek nyeregpontja van, ')
            print('mert a Hesse mátrixnak van negatív és pozitív sajátértéke.\n')
            draw_surface(func, float(roots[j][0]), float(roots[j][1]))
            print("\n")
            fig[0] = ['NaN']
        else:
            print("\n További vizsgálatra van szükség.\n")
            draw_surface(func, float(roots[j][0]), float(roots[j][1]))
            print("\n")
            fig[0] = ['NaN']

# FELADATOK

## Feladat 1. Legyen $f(x,y):=4x^2+9y^2+8x-36y+24$. Határozzuk meg $f$ lokális szélsőértékeit.

In [21]:
func = 4*x**2+9*y**2+8*x-36*y+24
draw_surface(func)
func_latex = latex(func)
varlist = [x, y]
dim = len(varlist)
grad_list = gradient(func, varlist)
grad_func = Matrix(grad_list)
print(f' '+func_latex+ ' szélsőértékei')
print("grad(f) : ")
grad_func
Eq_list = ['NaN' for _ in range(dim)]
roots = list(nonlinsolve(grad_list, varlist))
if len(roots) == 0:
    print(f'A '+func_latex+ ' függvénynek nincs lokális szélsőértéke')
    draw_surface(func)
    print("\n")
    fig[0] = ['NaN']
    exit
hesse_matrix = hessian(func, varlist)
print("Hess(f) : ")
hesse_matrix
print("A")
Eq_list = ['NaN' for _ in range(dim)]
for i in range(dim):
    Eq_list[i] = Eq(grad_func[i])
    Eq_list[i]
print("egyenletrendszerből adódik, hogy")
number_of_roots = len(roots)
for j in range(number_of_roots):
    print(f'\n {j + 1}. kritikus pont: ')
    for k in range(dim):
        print(f'               {varlist[k]} = {roots[j][k]}')
    print('\nA Hesse mátrix a kritikus pontban:')
    hesse = hesse_matrix.subs([(varlist[k], roots[j][k]) for k in range(len(varlist))])
    hesse
    print("A Hesse mátrix aldeterminánsai: ")
    aldet = ([hesse[0:m, 0:m].det() for m in range(1, dim+1)])
    aldet
    print("Előjelei: ")
    aldet_sig_num = np.sign(aldet)
    aldet_sig_str = ["Nan" for _ in range(dim) ]
    for m in range(0,dim):
        if aldet_sig_num[m] == -1:
            aldet_sig_str[m] = "-"
        elif aldet_sig_num[m] == 1:
            aldet_sig_str[m] = "+"
        else:
            aldet_sig_str[m] = "0"
    print("            ", aldet_sig_str)
    aldet_sig_str_odd = aldet_sig_str[0::2]
    aldet_sig_str_even = aldet_sig_str[1::2]
    if sum(aldet_sig_num) == dim:
        print(f'\n A ' +func_latex+ ' függvénynek lokális minimuma van.\n')
        draw_surface(func, float(roots[j][0]), float(roots[j][1]))
        print("\n")
        fig[0] = ['NaN']
    elif (aldet_sig_str_odd.count("-") == len(aldet_sig_str_odd)) \
        and (aldet_sig_str_even.count("+") == len(aldet_sig_str_even)):
        print(f'\n A ' +func_latex+ ' függvénynek lokális maximuma van.\n')
        draw_surface(func, float(roots[j][0]), float(roots[j][1]))
        print("\n")
        fig[0] = ['NaN']
    elif ("0" not in aldet_sig_str):
        print(f'\n A ' +func_latex+ ' függvénynek nyeregpontja van.\n')
        draw_surface(func, float(roots[j][0]), float(roots[j][1]))
        print("\n")
        fig[0] = ['NaN']
    else:
        hesse_float = np.array(hesse).astype(np.float64)
        eigvalues = np.linalg.eigvalsh(hesse_float)        
        eigvalues_sig = np.sign(eigvalues)
        if (-1 in eigvalues_sig) and (1 in eigvalues_sig):
            print(f'\n A ' +func_latex+ ' függvénynek nyeregpontja van, ')
            print('mert a Hesse mátrixnak van negatív és pozitív sajátértéke.\n')
            draw_surface(func, float(roots[j][0]), float(roots[j][1]))
            print("\n")
            fig[0] = ['NaN']
        else:
            print("\n További vizsgálatra van szükség.\n")
            draw_surface(func, float(roots[j][0]), float(roots[j][1]))
            print("\n")
            fig[0] = ['NaN']

 4 x^{2} + 8 x + 9 y^{2} - 36 y + 24 szélsőértékei
grad(f) : 


⎡ 8⋅x + 8 ⎤
⎢         ⎥
⎣18⋅y - 36⎦

Hess(f) : 


⎡8  0 ⎤
⎢     ⎥
⎣0  18⎦

A


...

egyenletrendszerből adódik, hogy

 1. kritikus pont: 
               x = -1
               y = 2

A Hesse mátrix a kritikus pontban:


⎡8  0 ⎤
⎢     ⎥
⎣0  18⎦

A Hesse mátrix aldeterminánsai: 


[8, 144]

Előjelei: 
             ['+', '+']

 A 4 x^{2} + 8 x + 9 y^{2} - 36 y + 24 függvénynek lokális minimuma van.



## Feladatok 2. Legyen $f(x,y):=\dfrac{1}{3}x^3+y^2+2xy-6x-3y+4$. Határozzuk meg $f$ lokális szélsőértékeit.

In [10]:
func = func = x**3/3 +y**2 + 2 * x * y - 6 * x - 3 * y + 4
func_latex = latex(func)
varlist = [x, y]
dim = len(varlist)
grad_list = gradient(func, varlist)
grad_func = Matrix(grad_list)
print(f' '+func_latex+ ' szélsőértékei')
print("grad(f) : ")
grad_func
Eq_list = ['NaN' for _ in range(dim)]
roots = list(nonlinsolve(grad_list, varlist))
if len(roots) == 0:
    print(f'A '+func_latex+ ' függvénynek nincs lokális szélsőértéke')
    draw_surface(func)
    print("\n")
    fig[0] = ['NaN']
    exit
hesse_matrix = hessian(func, varlist)
print("Hess(f) : ")
hesse_matrix
print("A")
Eq_list = ['NaN' for _ in range(dim)]
for i in range(dim):
    Eq_list[i] = Eq(grad_func[i])
    Eq_list[i]
print("egyenletrendszerből adódik, hogy")
number_of_roots = len(roots)
for j in range(number_of_roots):
    print(f'\n {j + 1}. kritikus pont: ')
    for k in range(dim):
        print(f'               {varlist[k]} = {roots[j][k]}')
    print('\nA Hesse mátrix a kritikus pontban:')
    hesse = hesse_matrix.subs([(varlist[k], roots[j][k]) for k in range(len(varlist))])
    hesse
    print("A Hesse mátrix aldeterminánsai: ")
    aldet = ([hesse[0:m, 0:m].det() for m in range(1, dim+1)])
    aldet
    print("Előjelei: ")
    aldet_sig_num = np.sign(aldet)
    aldet_sig_str = ["Nan" for _ in range(dim) ]
    for m in range(0,dim):
        if aldet_sig_num[m] == -1:
            aldet_sig_str[m] = "-"
        elif aldet_sig_num[m] == 1:
            aldet_sig_str[m] = "+"
        else:
            aldet_sig_str[m] = "0"
    print("            ", aldet_sig_str)
    aldet_sig_str_odd = aldet_sig_str[0::2]
    aldet_sig_str_even = aldet_sig_str[1::2]
    if sum(aldet_sig_num) == dim:
        print(f'\n A ' +func_latex+ ' függvénynek lokális minimuma van.\n')
        draw_surface(func, float(roots[j][0]), float(roots[j][1]))
        print("\n")
        fig[0] = ['NaN']
    elif (aldet_sig_str_odd.count("-") == len(aldet_sig_str_odd)) \
        and (aldet_sig_str_even.count("+") == len(aldet_sig_str_even)):
        print(f'\n A ' +func_latex+ ' függvénynek lokális maximuma van.\n')
        draw_surface(func, float(roots[j][0]), float(roots[j][1]))
        print("\n")
        fig[0] = ['NaN']
    elif ("0" not in aldet_sig_str):
        print(f'\n A ' +func_latex+ ' függvénynek nyeregpontja van.\n')
        draw_surface(func, float(roots[j][0]), float(roots[j][1]))
        print("\n")
        fig[0] = ['NaN']
    else:
        hesse_float = np.array(hesse).astype(np.float64)
        eigvalues = np.linalg.eigvalsh(hesse_float)        
        eigvalues_sig = np.sign(eigvalues)
        if (-1 in eigvalues_sig) and (1 in eigvalues_sig):
            print(f'\n A ' +func_latex+ ' függvénynek nyeregpontja van, ')
            print('mert a Hesse mátrixnak van negatív és pozitív sajátértéke.\n')
            draw_surface(func, float(roots[j][0]), float(roots[j][1]))
            print("\n")
            fig[0] = ['NaN']
        else:
            print("\n További vizsgálatra van szükség.\n")
            draw_surface(func, float(roots[j][0]), float(roots[j][1]))
            print("\n")
            fig[0] = ['NaN']

 \frac{x^{3}}{3} + 2 x y - 6 x + y^{2} - 3 y + 4 szélsőértékei
grad(f) : 


⎡ 2           ⎤
⎢x  + 2⋅y - 6 ⎥
⎢             ⎥
⎣2⋅x + 2⋅y - 3⎦

Hess(f) : 


⎡2⋅x  2⎤
⎢      ⎥
⎣ 2   2⎦

A


...




 2. kritikus pont: 
               x = 3
               y = -3/2

A Hesse mátrix a kritikus pontban:


⎡6  2⎤
⎢    ⎥
⎣2  2⎦

A Hesse mátrix aldeterminánsai: 


[6, 8]

Előjelei: 
             ['+', '+']

 A \frac{x^{3}}{3} + 2 x y - 6 x + y^{2} - 3 y + 4 függvénynek lokális minimuma van.



## Feladat 3. Legyen $f(x,y):=x^3+2xy-6x-4y^2$. Határozzuk meg $f$ lokális szélsőértékeit.

In [11]:
func = x**3 + 2 * x * y - 6 * x - 4 * y**2
func_latex = latex(func)
varlist = [x, y]
dim = len(varlist)
grad_list = gradient(func, varlist)
grad_func = Matrix(grad_list)
print(f' '+func_latex+ ' szélsőértékei')
print("grad(f) : ")
grad_func
Eq_list = ['NaN' for _ in range(dim)]
roots = list(nonlinsolve(grad_list, varlist))
if len(roots) == 0:
    print(f'A '+func_latex+ ' függvénynek nincs lokális szélsőértéke')
    draw_surface(func)
    print("\n")
    fig[0] = ['NaN']
    exit
hesse_matrix = hessian(func, varlist)
print("Hess(f) : ")
hesse_matrix
print("A")
Eq_list = ['NaN' for _ in range(dim)]
for i in range(dim):
    Eq_list[i] = Eq(grad_func[i])
    Eq_list[i]
print("egyenletrendszerből adódik, hogy")
number_of_roots = len(roots)
for j in range(number_of_roots):
    print(f'\n {j + 1}. kritikus pont: ')
    for k in range(dim):
        print(f'               {varlist[k]} = {roots[j][k]}')
    print('\nA Hesse mátrix a kritikus pontban:')
    hesse = hesse_matrix.subs([(varlist[k], roots[j][k]) for k in range(len(varlist))])
    hesse
    print("A Hesse mátrix aldeterminánsai: ")
    aldet = ([hesse[0:m, 0:m].det() for m in range(1, dim+1)])
    aldet
    print("Előjelei: ")
    aldet_sig_num = np.sign(aldet)
    aldet_sig_str = ["Nan" for _ in range(dim) ]
    for m in range(0,dim):
        if aldet_sig_num[m] == -1:
            aldet_sig_str[m] = "-"
        elif aldet_sig_num[m] == 1:
            aldet_sig_str[m] = "+"
        else:
            aldet_sig_str[m] = "0"
    print("            ", aldet_sig_str)
    aldet_sig_str_odd = aldet_sig_str[0::2]
    aldet_sig_str_even = aldet_sig_str[1::2]
    if sum(aldet_sig_num) == dim:
        print(f'\n A ' +func_latex+ ' függvénynek lokális minimuma van.\n')
        draw_surface(func, float(roots[j][0]), float(roots[j][1]))
        print("\n")
        fig[0] = ['NaN']
    elif (aldet_sig_str_odd.count("-") == len(aldet_sig_str_odd)) \
        and (aldet_sig_str_even.count("+") == len(aldet_sig_str_even)):
        print(f'\n A ' +func_latex+ ' függvénynek lokális maximuma van.\n')
        draw_surface(func, float(roots[j][0]), float(roots[j][1]))
        print("\n")
        fig[0] = ['NaN']
    elif ("0" not in aldet_sig_str):
        print(f'\n A ' +func_latex+ ' függvénynek nyeregpontja van.\n')
        draw_surface(func, float(roots[j][0]), float(roots[j][1]))
        print("\n")
        fig[0] = ['NaN']
    else:
        hesse_float = np.array(hesse).astype(np.float64)
        eigvalues = np.linalg.eigvalsh(hesse_float)        
        eigvalues_sig = np.sign(eigvalues)
        if (-1 in eigvalues_sig) and (1 in eigvalues_sig):
            print(f'\n A ' +func_latex+ ' függvénynek nyeregpontja van, ')
            print('mert a Hesse mátrixnak van negatív és pozitív sajátértéke.\n')
            draw_surface(func, float(roots[j][0]), float(roots[j][1]))
            print("\n")
            fig[0] = ['NaN']
        else:
            print("\n További vizsgálatra van szükség.\n")
            draw_surface(func, float(roots[j][0]), float(roots[j][1]))
            print("\n")
            fig[0] = ['NaN']

 x^{3} + 2 x y - 6 x - 4 y^{2} szélsőértékei
grad(f) : 


⎡   2          ⎤
⎢3⋅x  + 2⋅y - 6⎥
⎢              ⎥
⎣  2⋅x - 8⋅y   ⎦

Hess(f) : 


⎡6⋅x  2 ⎤
⎢       ⎥
⎣ 2   -8⎦

A


...




 2. kritikus pont: 
               x = 4/3
               y = 1/3

A Hesse mátrix a kritikus pontban:


⎡8  2 ⎤
⎢     ⎥
⎣2  -8⎦

A Hesse mátrix aldeterminánsai: 


[8, -68]

Előjelei: 
             ['+', '-']

 A x^{3} + 2 x y - 6 x - 4 y^{2} függvénynek nyeregpontja van.



## Feladat 4. Legyen $f(x,y):=2xy+3x+4y$. Határozzuk meg $f$ lokális szélsőértékeit.

In [12]:
func = 2 * x * y + 3 * x + 4 * y
func_latex = latex(func)
varlist = [x, y]
dim = len(varlist)
grad_list = gradient(func, varlist)
grad_func = Matrix(grad_list)
print(f' '+func_latex+ ' szélsőértékei')
print("grad(f) : ")
grad_func
Eq_list = ['NaN' for _ in range(dim)]
roots = list(nonlinsolve(grad_list, varlist))
if len(roots) == 0:
    print(f'A '+func_latex+ ' függvénynek nincs lokális szélsőértéke')
    draw_surface(func)
    print("\n")
    fig[0] = ['NaN']
    exit
hesse_matrix = hessian(func, varlist)
print("Hess(f) : ")
hesse_matrix
print("A")
Eq_list = ['NaN' for _ in range(dim)]
for i in range(dim):
    Eq_list[i] = Eq(grad_func[i])
    Eq_list[i]
print("egyenletrendszerből adódik, hogy")
number_of_roots = len(roots)
for j in range(number_of_roots):
    print(f'\n {j + 1}. kritikus pont: ')
    for k in range(dim):
        print(f'               {varlist[k]} = {roots[j][k]}')
    print('\nA Hesse mátrix a kritikus pontban:')
    hesse = hesse_matrix.subs([(varlist[k], roots[j][k]) for k in range(len(varlist))])
    hesse
    print("A Hesse mátrix aldeterminánsai: ")
    aldet = ([hesse[0:m, 0:m].det() for m in range(1, dim+1)])
    aldet
    print("Előjelei: ")
    aldet_sig_num = np.sign(aldet)
    aldet_sig_str = ["Nan" for _ in range(dim) ]
    for m in range(0,dim):
        if aldet_sig_num[m] == -1:
            aldet_sig_str[m] = "-"
        elif aldet_sig_num[m] == 1:
            aldet_sig_str[m] = "+"
        else:
            aldet_sig_str[m] = "0"
    print("            ", aldet_sig_str)
    aldet_sig_str_odd = aldet_sig_str[0::2]
    aldet_sig_str_even = aldet_sig_str[1::2]
    if sum(aldet_sig_num) == dim:
        print(f'\n A ' +func_latex+ ' függvénynek lokális minimuma van.\n')
        draw_surface(func, float(roots[j][0]), float(roots[j][1]))
        print("\n")
        fig[0] = ['NaN']
    elif (aldet_sig_str_odd.count("-") == len(aldet_sig_str_odd)) \
        and (aldet_sig_str_even.count("+") == len(aldet_sig_str_even)):
        print(f'\n A ' +func_latex+ ' függvénynek lokális maximuma van.\n')
        draw_surface(func, float(roots[j][0]), float(roots[j][1]))
        print("\n")
        fig[0] = ['NaN']
    elif ("0" not in aldet_sig_str):
        print(f'\n A ' +func_latex+ ' függvénynek nyeregpontja van.\n')
        draw_surface(func, float(roots[j][0]), float(roots[j][1]))
        print("\n")
        fig[0] = ['NaN']
    else:
        hesse_float = np.array(hesse).astype(np.float64)
        eigvalues = np.linalg.eigvalsh(hesse_float)        
        eigvalues_sig = np.sign(eigvalues)
        if (-1 in eigvalues_sig) and (1 in eigvalues_sig):
            print(f'\n A ' +func_latex+ ' függvénynek nyeregpontja van, ')
            print('mert a Hesse mátrixnak van negatív és pozitív sajátértéke.\n')
            draw_surface(func, float(roots[j][0]), float(roots[j][1]))
            print("\n")
            fig[0] = ['NaN']
        else:
            print("\n További vizsgálatra van szükség.\n")
            draw_surface(func, float(roots[j][0]), float(roots[j][1]))
            print("\n")
            fig[0] = ['NaN']

 2 x y + 3 x + 4 y szélsőértékei
grad(f) : 


⎡2⋅y + 3⎤
⎢       ⎥
⎣2⋅x + 4⎦

Hess(f) : 


⎡0  2⎤
⎢    ⎥
⎣2  0⎦

A


...

egyenletrendszerből adódik, hogy

 1. kritikus pont: 
               x = -2
               y = -3/2

A Hesse mátrix a kritikus pontban:


⎡0  2⎤
⎢    ⎥
⎣2  0⎦

A Hesse mátrix aldeterminánsai: 


[0, -4]

Előjelei: 
             ['0', '-']

 A 2 x y + 3 x + 4 y függvénynek nyeregpontja van, 
mert a Hesse mátrixnak van negatív és pozitív sajátértéke.



## Feladat 5. Legyen $f(x,y):=3x^2-2xy+y^2-8y$. Határozzuk meg $f$ lokális szélsőértékeit.

In [13]:
func = 3 * x**2 - 2 * x * y + y**2 - 8 * y
func_latex = latex(func)
varlist = [x, y]
dim = len(varlist)
grad_list = gradient(func, varlist)
grad_func = Matrix(grad_list)
print(f' '+func_latex+ ' szélsőértékei')
print("grad(f) : ")
grad_func
Eq_list = ['NaN' for _ in range(dim)]
roots = list(nonlinsolve(grad_list, varlist))
if len(roots) == 0:
    print(f'A '+func_latex+ ' függvénynek nincs lokális szélsőértéke')
    draw_surface(func)
    print("\n")
    fig[0] = ['NaN']
    exit
hesse_matrix = hessian(func, varlist)
print("Hess(f) : ")
hesse_matrix
print("A")
Eq_list = ['NaN' for _ in range(dim)]
for i in range(dim):
    Eq_list[i] = Eq(grad_func[i])
    Eq_list[i]
print("egyenletrendszerből adódik, hogy")
number_of_roots = len(roots)
for j in range(number_of_roots):
    print(f'\n {j + 1}. kritikus pont: ')
    for k in range(dim):
        print(f'               {varlist[k]} = {roots[j][k]}')
    print('\nA Hesse mátrix a kritikus pontban:')
    hesse = hesse_matrix.subs([(varlist[k], roots[j][k]) for k in range(len(varlist))])
    hesse
    print("A Hesse mátrix aldeterminánsai: ")
    aldet = ([hesse[0:m, 0:m].det() for m in range(1, dim+1)])
    aldet
    print("Előjelei: ")
    aldet_sig_num = np.sign(aldet)
    aldet_sig_str = ["Nan" for _ in range(dim) ]
    for m in range(0,dim):
        if aldet_sig_num[m] == -1:
            aldet_sig_str[m] = "-"
        elif aldet_sig_num[m] == 1:
            aldet_sig_str[m] = "+"
        else:
            aldet_sig_str[m] = "0"
    print("            ", aldet_sig_str)
    aldet_sig_str_odd = aldet_sig_str[0::2]
    aldet_sig_str_even = aldet_sig_str[1::2]
    if sum(aldet_sig_num) == dim:
        print(f'\n A ' +func_latex+ ' függvénynek lokális minimuma van.\n')
        draw_surface(func, float(roots[j][0]), float(roots[j][1]))
        print("\n")
        fig[0] = ['NaN']
    elif (aldet_sig_str_odd.count("-") == len(aldet_sig_str_odd)) \
        and (aldet_sig_str_even.count("+") == len(aldet_sig_str_even)):
        print(f'\n A ' +func_latex+ ' függvénynek lokális maximuma van.\n')
        draw_surface(func, float(roots[j][0]), float(roots[j][1]))
        print("\n")
        fig[0] = ['NaN']
    elif ("0" not in aldet_sig_str):
        print(f'\n A ' +func_latex+ ' függvénynek nyeregpontja van.\n')
        draw_surface(func, float(roots[j][0]), float(roots[j][1]))
        print("\n")
        fig[0] = ['NaN']
    else:
        hesse_float = np.array(hesse).astype(np.float64)
        eigvalues = np.linalg.eigvalsh(hesse_float)        
        eigvalues_sig = np.sign(eigvalues)
        if (-1 in eigvalues_sig) and (1 in eigvalues_sig):
            print(f'\n A ' +func_latex+ ' függvénynek nyeregpontja van, ')
            print('mert a Hesse mátrixnak van negatív és pozitív sajátértéke.\n')
            draw_surface(func, float(roots[j][0]), float(roots[j][1]))
            print("\n")
            fig[0] = ['NaN']
        else:
            print("\n További vizsgálatra van szükség.\n")
            draw_surface(func, float(roots[j][0]), float(roots[j][1]))
            print("\n")
            fig[0] = ['NaN']

 3 x^{2} - 2 x y + y^{2} - 8 y szélsőértékei
grad(f) : 


⎡  6⋅x - 2⋅y   ⎤
⎢              ⎥
⎣-2⋅x + 2⋅y - 8⎦

Hess(f) : 


⎡6   -2⎤
⎢      ⎥
⎣-2  2 ⎦

A


...

egyenletrendszerből adódik, hogy

 1. kritikus pont: 
               x = 2
               y = 6

A Hesse mátrix a kritikus pontban:


⎡6   -2⎤
⎢      ⎥
⎣-2  2 ⎦

A Hesse mátrix aldeterminánsai: 


[6, 8]

Előjelei: 
             ['+', '+']

 A 3 x^{2} - 2 x y + y^{2} - 8 y függvénynek lokális minimuma van.



## Feladat 6. Legyen $f(x,y):=y^2+xy+3y+2x+3$. Határozzuk meg $f$ lokális szélsőértékeit.

In [14]:
func = y**2 + x * y + 3 * y + 2 * x + 3
func_latex = latex(func)
varlist = [x, y]
dim = len(varlist)
grad_list = gradient(func, varlist)
grad_func = Matrix(grad_list)
print(f' '+func_latex+ ' szélsőértékei')
print("grad(f) : ")
grad_func
Eq_list = ['NaN' for _ in range(dim)]
roots = list(nonlinsolve(grad_list, varlist))
if len(roots) == 0:
    print(f'A '+func_latex+ ' függvénynek nincs lokális szélsőértéke')
    draw_surface(func)
    print("\n")
    fig[0] = ['NaN']
    exit
hesse_matrix = hessian(func, varlist)
print("Hess(f) : ")
hesse_matrix
print("A")
Eq_list = ['NaN' for _ in range(dim)]
for i in range(dim):
    Eq_list[i] = Eq(grad_func[i])
    Eq_list[i]
print("egyenletrendszerből adódik, hogy")
number_of_roots = len(roots)
for j in range(number_of_roots):
    print(f'\n {j + 1}. kritikus pont: ')
    for k in range(dim):
        print(f'               {varlist[k]} = {roots[j][k]}')
    print('\nA Hesse mátrix a kritikus pontban:')
    hesse = hesse_matrix.subs([(varlist[k], roots[j][k]) for k in range(len(varlist))])
    hesse
    print("A Hesse mátrix aldeterminánsai: ")
    aldet = ([hesse[0:m, 0:m].det() for m in range(1, dim+1)])
    aldet
    print("Előjelei: ")
    aldet_sig_num = np.sign(aldet)
    aldet_sig_str = ["Nan" for _ in range(dim) ]
    for m in range(0,dim):
        if aldet_sig_num[m] == -1:
            aldet_sig_str[m] = "-"
        elif aldet_sig_num[m] == 1:
            aldet_sig_str[m] = "+"
        else:
            aldet_sig_str[m] = "0"
    print("            ", aldet_sig_str)
    aldet_sig_str_odd = aldet_sig_str[0::2]
    aldet_sig_str_even = aldet_sig_str[1::2]
    if sum(aldet_sig_num) == dim:
        print(f'\n A ' +func_latex+ ' függvénynek lokális minimuma van.\n')
        draw_surface(func, float(roots[j][0]), float(roots[j][1]))
        print("\n")
        fig[0] = ['NaN']
    elif (aldet_sig_str_odd.count("-") == len(aldet_sig_str_odd)) \
        and (aldet_sig_str_even.count("+") == len(aldet_sig_str_even)):
        print(f'\n A ' +func_latex+ ' függvénynek lokális maximuma van.\n')
        draw_surface(func, float(roots[j][0]), float(roots[j][1]))
        print("\n")
        fig[0] = ['NaN']
    elif ("0" not in aldet_sig_str):
        print(f'\n A ' +func_latex+ ' függvénynek nyeregpontja van.\n')
        draw_surface(func, float(roots[j][0]), float(roots[j][1]))
        print("\n")
        fig[0] = ['NaN']
    else:
        hesse_float = np.array(hesse).astype(np.float64)
        eigvalues = np.linalg.eigvalsh(hesse_float)        
        eigvalues_sig = np.sign(eigvalues)
        if (-1 in eigvalues_sig) and (1 in eigvalues_sig):
            print(f'\n A ' +func_latex+ ' függvénynek nyeregpontja van, ')
            print('mert a Hesse mátrixnak van negatív és pozitív sajátértéke.\n')
            draw_surface(func, float(roots[j][0]), float(roots[j][1]))
            print("\n")
            fig[0] = ['NaN']
        else:
            print("\n További vizsgálatra van szükség.\n")
            draw_surface(func, float(roots[j][0]), float(roots[j][1]))
            print("\n")
            fig[0] = ['NaN']

 x y + 2 x + y^{2} + 3 y + 3 szélsőértékei
grad(f) : 


⎡   y + 2   ⎤
⎢           ⎥
⎣x + 2⋅y + 3⎦

Hess(f) : 


⎡0  1⎤
⎢    ⎥
⎣1  2⎦

A


...

egyenletrendszerből adódik, hogy

 1. kritikus pont: 
               x = 1
               y = -2

A Hesse mátrix a kritikus pontban:


⎡0  1⎤
⎢    ⎥
⎣1  2⎦

A Hesse mátrix aldeterminánsai: 


[0, -1]

Előjelei: 
             ['0', '-']

 A x y + 2 x + y^{2} + 3 y + 3 függvénynek nyeregpontja van, 
mert a Hesse mátrixnak van negatív és pozitív sajátértéke.



## Feladat 7. Legyen $f(x,y):=x^2+2y^2-x^2y$. Határozzuk meg $f$ lokális szélsőértékeit.

In [15]:
func = x**2 + 2 * y**2 - x**2 * y
func_latex = latex(func)
varlist = [x, y]
dim = len(varlist)
grad_list = gradient(func, varlist)
grad_func = Matrix(grad_list)
print(f' '+func_latex+ ' szélsőértékei')
print("grad(f) : ")
grad_func
Eq_list = ['NaN' for _ in range(dim)]
roots = list(nonlinsolve(grad_list, varlist))
if len(roots) == 0:
    print(f'A '+func_latex+ ' függvénynek nincs lokális szélsőértéke')
    draw_surface(func)
    print("\n")
    fig[0] = ['NaN']
    exit
hesse_matrix = hessian(func, varlist)
print("Hess(f) : ")
hesse_matrix
print("A")
Eq_list = ['NaN' for _ in range(dim)]
for i in range(dim):
    Eq_list[i] = Eq(grad_func[i])
    Eq_list[i]
print("egyenletrendszerből adódik, hogy")
number_of_roots = len(roots)
for j in range(number_of_roots):
    print(f'\n {j + 1}. kritikus pont: ')
    for k in range(dim):
        print(f'               {varlist[k]} = {roots[j][k]}')
    print('\nA Hesse mátrix a kritikus pontban:')
    hesse = hesse_matrix.subs([(varlist[k], roots[j][k]) for k in range(len(varlist))])
    hesse
    print("A Hesse mátrix aldeterminánsai: ")
    aldet = ([hesse[0:m, 0:m].det() for m in range(1, dim+1)])
    aldet
    print("Előjelei: ")
    aldet_sig_num = np.sign(aldet)
    aldet_sig_str = ["Nan" for _ in range(dim) ]
    for m in range(0,dim):
        if aldet_sig_num[m] == -1:
            aldet_sig_str[m] = "-"
        elif aldet_sig_num[m] == 1:
            aldet_sig_str[m] = "+"
        else:
            aldet_sig_str[m] = "0"
    print("            ", aldet_sig_str)
    aldet_sig_str_odd = aldet_sig_str[0::2]
    aldet_sig_str_even = aldet_sig_str[1::2]
    if sum(aldet_sig_num) == dim:
        print(f'\n A ' +func_latex+ ' függvénynek lokális minimuma van.\n')
        draw_surface(func, float(roots[j][0]), float(roots[j][1]))
        print("\n")
        fig[0] = ['NaN']
    elif (aldet_sig_str_odd.count("-") == len(aldet_sig_str_odd)) \
        and (aldet_sig_str_even.count("+") == len(aldet_sig_str_even)):
        print(f'\n A ' +func_latex+ ' függvénynek lokális maximuma van.\n')
        draw_surface(func, float(roots[j][0]), float(roots[j][1]))
        print("\n")
        fig[0] = ['NaN']
    elif ("0" not in aldet_sig_str):
        print(f'\n A ' +func_latex+ ' függvénynek nyeregpontja van.\n')
        draw_surface(func, float(roots[j][0]), float(roots[j][1]))
        print("\n")
        fig[0] = ['NaN']
    else:
        hesse_float = np.array(hesse).astype(np.float64)
        eigvalues = np.linalg.eigvalsh(hesse_float)        
        eigvalues_sig = np.sign(eigvalues)
        if (-1 in eigvalues_sig) and (1 in eigvalues_sig):
            print(f'\n A ' +func_latex+ ' függvénynek nyeregpontja van, ')
            print('mert a Hesse mátrixnak van negatív és pozitív sajátértéke.\n')
            draw_surface(func, float(roots[j][0]), float(roots[j][1]))
            print("\n")
            fig[0] = ['NaN']
        else:
            print("\n További vizsgálatra van szükség.\n")
            draw_surface(func, float(roots[j][0]), float(roots[j][1]))
            print("\n")
            fig[0] = ['NaN']

 - x^{2} y + x^{2} + 2 y^{2} szélsőértékei
grad(f) : 


⎡-2⋅x⋅y + 2⋅x⎤
⎢            ⎥
⎢    2       ⎥
⎣ - x  + 4⋅y ⎦

Hess(f) : 


⎡-2⋅y + 2  -2⋅x⎤
⎢              ⎥
⎣  -2⋅x     4  ⎦

A


...




 3. kritikus pont: 
               x = 2
               y = 1

A Hesse mátrix a kritikus pontban:


⎡0   -4⎤
⎢      ⎥
⎣-4  4 ⎦

A Hesse mátrix aldeterminánsai: 


[0, -16]

Előjelei: 
             ['0', '-']

 A - x^{2} y + x^{2} + 2 y^{2} függvénynek nyeregpontja van, 
mert a Hesse mátrixnak van negatív és pozitív sajátértéke.



## Feladat 8. Legyen $f(x,y):=x^2+10xy+y^2$. Határozzuk meg $f$ lokális szélsőértékeit.

In [16]:
func = x**2 + 10 * x * y + y**2
func_latex = latex(func)
varlist = [x, y]
dim = len(varlist)
grad_list = gradient(func, varlist)
grad_func = Matrix(grad_list)
print(f' '+func_latex+ ' szélsőértékei')
print("grad(f) : ")
grad_func
Eq_list = ['NaN' for _ in range(dim)]
roots = list(nonlinsolve(grad_list, varlist))
if len(roots) == 0:
    print(f'A '+func_latex+ ' függvénynek nincs lokális szélsőértéke')
    draw_surface(func)
    print("\n")
    fig[0] = ['NaN']
    exit
hesse_matrix = hessian(func, varlist)
print("Hess(f) : ")
hesse_matrix
print("A")
Eq_list = ['NaN' for _ in range(dim)]
for i in range(dim):
    Eq_list[i] = Eq(grad_func[i])
    Eq_list[i]
print("egyenletrendszerből adódik, hogy")
number_of_roots = len(roots)
for j in range(number_of_roots):
    print(f'\n {j + 1}. kritikus pont: ')
    for k in range(dim):
        print(f'               {varlist[k]} = {roots[j][k]}')
    print('\nA Hesse mátrix a kritikus pontban:')
    hesse = hesse_matrix.subs([(varlist[k], roots[j][k]) for k in range(len(varlist))])
    hesse
    print("A Hesse mátrix aldeterminánsai: ")
    aldet = ([hesse[0:m, 0:m].det() for m in range(1, dim+1)])
    aldet
    print("Előjelei: ")
    aldet_sig_num = np.sign(aldet)
    aldet_sig_str = ["Nan" for _ in range(dim) ]
    for m in range(0,dim):
        if aldet_sig_num[m] == -1:
            aldet_sig_str[m] = "-"
        elif aldet_sig_num[m] == 1:
            aldet_sig_str[m] = "+"
        else:
            aldet_sig_str[m] = "0"
    print("            ", aldet_sig_str)
    aldet_sig_str_odd = aldet_sig_str[0::2]
    aldet_sig_str_even = aldet_sig_str[1::2]
    if sum(aldet_sig_num) == dim:
        print(f'\n A ' +func_latex+ ' függvénynek lokális minimuma van.\n')
        draw_surface(func, float(roots[j][0]), float(roots[j][1]))
        print("\n")
        fig[0] = ['NaN']
    elif (aldet_sig_str_odd.count("-") == len(aldet_sig_str_odd)) \
        and (aldet_sig_str_even.count("+") == len(aldet_sig_str_even)):
        print(f'\n A ' +func_latex+ ' függvénynek lokális maximuma van.\n')
        draw_surface(func, float(roots[j][0]), float(roots[j][1]))
        print("\n")
        fig[0] = ['NaN']
    elif ("0" not in aldet_sig_str):
        print(f'\n A ' +func_latex+ ' függvénynek nyeregpontja van.\n')
        draw_surface(func, float(roots[j][0]), float(roots[j][1]))
        print("\n")
        fig[0] = ['NaN']
    else:
        hesse_float = np.array(hesse).astype(np.float64)
        eigvalues = np.linalg.eigvalsh(hesse_float)        
        eigvalues_sig = np.sign(eigvalues)
        if (-1 in eigvalues_sig) and (1 in eigvalues_sig):
            print(f'\n A ' +func_latex+ ' függvénynek nyeregpontja van, ')
            print('mert a Hesse mátrixnak van negatív és pozitív sajátértéke.\n')
            draw_surface(func, float(roots[j][0]), float(roots[j][1]))
            print("\n")
            fig[0] = ['NaN']
        else:
            print("\n További vizsgálatra van szükség.\n")
            draw_surface(func, float(roots[j][0]), float(roots[j][1]))
            print("\n")
            fig[0] = ['NaN']

 x^{2} + 10 x y + y^{2} szélsőértékei
grad(f) : 


⎡2⋅x + 10⋅y⎤
⎢          ⎥
⎣10⋅x + 2⋅y⎦

Hess(f) : 


⎡2   10⎤
⎢      ⎥
⎣10  2 ⎦

A


...

egyenletrendszerből adódik, hogy

 1. kritikus pont: 
               x = 0
               y = 0

A Hesse mátrix a kritikus pontban:


⎡2   10⎤
⎢      ⎥
⎣10  2 ⎦

A Hesse mátrix aldeterminánsai: 


[2, -96]

Előjelei: 
             ['+', '-']

 A x^{2} + 10 x y + y^{2} függvénynek nyeregpontja van.



In [17]:
%%html
<script src="https://cdn.rawgit.com/parente/4c3e6936d0d7a46fd071/raw/65b816fb9bdd3c28b4ddf3af602bfd6015486383/code_toggle.js"></script>